# Comparative analyses of machine learning-based models
This juptyer notebook goes through the compiling, training, testing and visualisation of machine learning models used for bacterial taxonomy classification using the curated 16S rRNA sequence datasets.

## Loading modules and training variables

In [1]:
# LOADING PACKAGES
import numpy as np
import pandas as pd
import sys
from varname import argname, nameof
import time
import string
import os
import keras
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import f1_score, matthews_corrcoef, accuracy_score

from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten, Masking, Dot, Add, BatchNormalization
from keras.layers import MaxPooling1D, AveragePooling1D, Conv1D, Reshape
from keras.layers import TimeDistributed, LSTM, Bidirectional

import wandb
from wandb.keras import WandbCallback

LR, BATCH_SIZE, EPOCHS, MAX_LEN, INPUT_SHAPE_RNN, INPUT_SHAPE_4_MER, INPUT_SHAPE_7_MER = 0.001, 8, 3, 2000, (2000, 4), (625, 1), (78125, 1)

print('Packages loaded!')

Packages loaded!


## Loading and preprocessing the data

In [36]:
# LOADING THE NON-AUGMENTED AND AUGMENTED DATASETS
train_na = pd.read_csv('df_train_0.csv')
val_na = pd.read_csv('df_val_0.csv')
test_na = pd.read_csv('df_test_0.csv')
# ------------------------------------------------
train_a = pd.read_csv('df_train_1.csv')
val_a = pd.read_csv('df_val_1.csv')
# ------------------------------------------------
print('Datasets loaded!')

Datasets loaded!


### Generating k-mers
For use in the Convolutional Neural Networks (CNN), the sequences are processed into a frequency table of k-mers.

In [4]:
from itertools import product
# define all possible k-mers
alphabet = "AGTCN"
four_mers = [''.join(chars) for chars in product(*(4*(alphabet,)))]
seven_mers = [''.join(chars) for chars in product(*(7*(alphabet,)))]

def one_hot_k(sequence, kmers):
    k = len(kmers[0])
    # define a counter dictionary
    kmer_dict = dict.fromkeys(kmers, 0)

    # standardize the sequence
    # by replacing U with T and all ambiguous bases with N
    sequence = sequence.replace('U', 'T').replace('Y', 'N').replace('R', 'N').replace('W', 'N').replace('S', 'N').replace('K', 'N').replace('M', 'N').replace('D', 'N').replace('V', 'N').replace('H', 'N').replace('B', 'N').replace('X', 'N').replace('-', 'N')
    # count every k-mer in the sequence
    for i in range(0, len(sequence) -k+1):
        kmer_dict[sequence[i:i+k]] += 1

    # k-mer frequency array from the dictionary values
    k_array = np.array(list(kmer_dict.values()))
    # normalizing the array by dividing every value with the highest count value
    k_array = k_array / np.amax(k_array)
    return k_array

### One-hot-encoding the sequences
For use in the various Recurrent Neural Networks (RNN), the nucleotide sequences are processed into a one-hot-encoded format.

In [5]:
# Dictionary without consideration of mutation rate (AGTC)
one_hot_dict0 = {'A': [1.,0.,0.,0.], 'G':[0.,1.,0.,0.], 'T':[0.,0.,1.,0.], 'U':[0.,0.,1.,0.], 'C':[0.,0.,0.,1.], 'Y':[0.,0.,0.5,0.5], 'R':[0.5,0.5,0.,0.], 'W':[0.5,0.,0.5,0.], 'S':[0.,0.5,0.,0.5], 'K':[0.,0.5,0.5,0.], 'M':[0.5,0.,0.,0.5], 'D':[0.33,0.33,0.33,0.], 'V':[0.33,0.33,0.,0.33], 'H':[0.33,0.,0.33,0.33], 'B':[0.,0.33,0.33,0.33], 'X':[0.25,0.25,0.25,0.25], 'N':[0.25,0.25,0.25,0.25], '-':[0.,0.,0.,0.]}
# Dictionary with consideration of mutation rate (AGTC)
one_hot_dict1 = {'A': [1.,0.,-0.5,-0.5], 'G':[0.,1.,-0.5,-0.5], 'T':[-0.5,-0.5,1.,0.], 'U':[-0.5,-0.5,1.,0.], 'C':[-0.5,-0.5,0.,1.], 'Y':[-0.5,-0.5,0.5,0.5], 'R':[0.5,0.5,-0.5,-0.5], 'W':[0.5,-0.5,0.5,-0.5], 'S':[-0.5,0.5,-0.5,0.5], 'K':[-0.5,0.5,0.5,-0.5], 'M':[0.5,-0.5,-0.5,0.5], 'D':[0.33,0.33,0.33,-1.], 'V':[0.33,0.33,-1.,0.33], 'H':[0.33,-1.,.33,0.33], 'B':[-1.,0.33,0.33,0.33], 'X':[0.,0.,0.,0.], 'N':[0.,0.,0.,0.], '-':[0.,0.,0.,0.]}

def one_hot_seq(sequence, one_hot_dict, MAX_LEN=MAX_LEN):
    # padding the sequences to a fixed length
	sequence += '-'*(MAX_LEN - len(sequence))
    # generating list of one-hot-lists using the dictionary
	onehot_encoded = [one_hot_dict[nucleotide] for nucleotide in sequence]
    # returning the list of lists as a numpy array
	return np.array(onehot_encoded)

### One-hot-encoding the labels
For use in the deep learning models, the labels are processed into a one-hot-encoding format. To achieve this, every unique label is first encoded to a numerical value.

In [6]:
def get_taxon_dict(df, taxon):
    # listing all unique taxon labels
    taxon_list = list(df[taxon].unique())

    # generating a dictionary to associate every unique taxon to a number
    taxon_dict = dict(zip(taxon_list, range(0, len(taxon_list))))
    # and the reversed dictionary as a lookup table
    taxon_dict_lookup = {v: k for k, v in taxon_dict.items()}

    return taxon_dict, taxon_dict_lookup

## Generating the input sequences and labels
The encoding methods for nucleotide sequences (x) defined earlier are now applied. The labels (y) are one-hot-encoded using the to_categorical function in keras_utils, thereby, converting the data into the correct format for feeding it to the deep learning models.

Every array is saved to reduce memory requirements down the road.

### Encoding the sequences into k-mer counts and one-hot-encoded sequences

In [8]:
# ENCODING SEQUENCES for the CNN models in 2 processing variations
# FOR CNN  |  with 4-mer
x_train_CNN_na4 = np.array(train_na['Sequence'].apply(lambda x: one_hot_k(x, four_mers)).tolist())
np.save(f'arrays/CNN/{nameof(x_train_CNN_na4)}', x_train_CNN_na4)

# x_train_CNN_a4 = np.array(train_a['Sequence'].apply(lambda x: one_hot_k(x, four_mers)).tolist())
# np.save(f'arrays/CNN/{nameof(x_train_CNN_a4)}', x_train_CNN_a4)

x_test_CNN_na4 = np.array(test_na['Sequence'].apply(lambda x: one_hot_k(x, four_mers)).tolist())
np.save(f'arrays/CNN/{nameof(x_test_CNN_na4)}', x_test_CNN_na4)

dataval_CNN_na4 = np.array(val_na['Sequence'].apply(lambda x: one_hot_k(x, four_mers)).tolist())
np.save(f'arrays/CNN/{nameof(dataval_CNN_na4)}', dataval_CNN_na4)

# dataval_CNN_a4 = np.array(val_a['Sequence'].apply(lambda x: one_hot_k(x, four_mers)).tolist())
# np.save(f'arrays/CNN/{nameof(dataval_CNN_a4)}', dataval_CNN_a4)
print('4-mer complete')
# -----------------------------------------------------------------------------------------------------
# FOR CNN  |  with 7-mer
x_train_CNN_na7 = np.array(train_na['Sequence'].apply(lambda x: one_hot_k(x, seven_mers)).tolist())
np.save(f'arrays/CNN/{nameof(x_train_CNN_na7)}', x_train_CNN_na7)

x_train_CNN_a7 = np.array(train_a['Sequence'].apply(lambda x: one_hot_k(x, seven_mers)).tolist())
np.save(f'arrays/CNN/{nameof(x_train_CNN_a7)}', x_train_CNN_a7)

x_test_CNN_na7 = np.array(test_na['Sequence'].apply(lambda x: one_hot_k(x, seven_mers)).tolist())
np.save(f'arrays/CNN/{nameof(x_test_CNN_na7)}', x_test_CNN_na7)

dataval_CNN_na7 = np.array(val_na['Sequence'].apply(lambda x: one_hot_k(x, seven_mers)).tolist())
np.save(f'arrays/CNN/{nameof(dataval_CNN_na7)}', dataval_CNN_na7)

dataval_CNN_a7 = np.array(val_a['Sequence'].apply(lambda x: one_hot_k(x, seven_mers)).tolist())
np.save(f'arrays/CNN/{nameof(dataval_CNN_a7)}', dataval_CNN_a7)
print('7-mer complete')
# -----------------------------------------------------------------------------------------------------
print('CNN sequences complete')

4-mer complete
7-mer complete
CNN sequences complete


In [9]:
# ENCODING SEQUENCES for the RNN models in 2 processing variations
# FOR RNN  |  with regular one-hot-encoding
x_train_RNN_na0 = np.array(train_na['Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict0)).tolist())
np.save(f'arrays/RNN/{nameof(x_train_RNN_na0)}', x_train_RNN_na0)

# x_train_RNN_a0 = np.array(train_a['Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict0)).tolist())
# np.save(f'arrays/RNN/{nameof(x_train_RNN_a0)}', x_train_RNN_a0)

x_test_RNN_na0 = np.array(test_na['Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict0)).tolist())
np.save(f'arrays/RNN/{nameof(x_test_RNN_na0)}', x_test_RNN_na0)

dataval_RNN_na0 = np.array(val_na['Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict0)).tolist())
np.save(f'arrays/RNN/{nameof(dataval_RNN_na0)}', dataval_RNN_na0)

# dataval_RNN_a0 = np.array(val_a['Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict0)).tolist())
# np.save(f'arrays/RNN/{nameof(dataval_RNN_a0)}', dataval_RNN_a0)
print('Regular one-hot-encoding complete')
# -----------------------------------------------------------------------------------------------------
# FOR RNN  |  with matation rate adjusted one-hot-encoding
x_train_RNN_na1 = np.array(train_na['Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict1)).tolist())
np.save(f'arrays/RNN/{nameof(x_train_RNN_na1)}', x_train_RNN_na1)

x_train_RNN_a1 = np.array(train_a['Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict1)).tolist())
np.save(f'arrays/RNN/{nameof(x_train_RNN_a1)}', x_train_RNN_a1)

x_test_RNN_na1 = np.array(test_na['Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict1)).tolist())
np.save(f'arrays/RNN/{nameof(x_test_RNN_na1)}', x_test_RNN_na1)

dataval_RNN_na1 = np.array(val_na['Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict1)).tolist())
np.save(f'arrays/RNN/{nameof(dataval_RNN_na1)}', dataval_RNN_na1)

dataval_RNN_a1 = np.array(val_a['Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict1)).tolist())
np.save(f'arrays/RNN/{nameof(dataval_RNN_a1)}', dataval_RNN_a1)
print('Mutations rate adjusted one-hot-encoding complete')
# -----------------------------------------------------------------------------------------------------
print('RNN sequences complete')

Regular one-hot-encoding complete
Mutations rate adjusted one-hot-encoding complete
RNN sequences complete


### Encoding the labels into a one-hot-encoded format at Family level

In [10]:
taxon = 'Family'
taxon_dict = get_taxon_dict(test_na, taxon)[0]

# Associate every entry's label in the df to a number 
#   using the dictionary & one-hot encode the numerical labels
y_train_fam_na = to_categorical(y=train_na[taxon].map(taxon_dict).astype(np.float32))
np.save(f'arrays/family/{nameof(y_train_fam_na)}', y_train_fam_na)

# y_train_fam_a = to_categorical(y=train_a[taxon].map(taxon_dict).astype(np.float32))
# np.save(f'arrays/family/{nameof(y_train_fam_a)}', y_train_fam_a)

y_test_fam_na = to_categorical(y=test_na[taxon].map(taxon_dict).astype(np.float32))
np.save(f'arrays/family/{nameof(y_test_fam_na)}', y_test_fam_na)

labelsval_fam_na = to_categorical(y=val_na[taxon].map(taxon_dict).astype(np.float32))
np.save(f'arrays/family/{nameof(labelsval_fam_na)}', labelsval_fam_na)

# labelsval_fam_a = to_categorical(y=val_a[taxon].map(taxon_dict).astype(np.float32))
# np.save(f'arrays/family/{nameof(labelsval_fam_a)}', labelsval_fam_a)
print('Family label arrays generated')
# ------------------------------------------------------------------------------------
fam_count = train_na[taxon].nunique()
print(f'The number of unique family labels: {fam_count}')

Labels complete
Family train/test/val arrays generated
The number of unique family labels: 349


### Encoding the labels into a one-hot-encoded format at Genus level

In [11]:
taxon = 'Genus'
taxon_dict = get_taxon_dict(test_na, taxon)[0]

y_train_gen_na = to_categorical(y=train_na[taxon].map(taxon_dict).astype(np.float32))
np.save(f'arrays/genus/{nameof(y_train_gen_na)}', y_train_gen_na)

y_train_gen_a = to_categorical(y=train_a[taxon].map(taxon_dict).astype(np.float32))
np.save(f'arrays/genus/{nameof(y_train_gen_a)}', y_train_gen_a)

y_test_gen_na = to_categorical(y=test_na[taxon].map(taxon_dict).astype(np.float32))
np.save(f'arrays/genus/{nameof(y_test_gen_na)}', y_test_gen_na)

labelsval_gen_na = to_categorical(y=val_na[taxon].map(taxon_dict).astype(np.float32))
np.save(f'arrays/genus/{nameof(labelsval_gen_na)}', labelsval_gen_na)

labelsval_gen_a = to_categorical(y=val_a[taxon].map(taxon_dict).astype(np.float32))
np.save(f'arrays/genus/{nameof(labelsval_gen_a)}', labelsval_gen_a)
print('Genus label arrays generated')
# ------------------------------------------------------------------------------------
gen_count = train_na[taxon].nunique()
print(f'The number of unique genus labels: {gen_count}')

Genus train/test/val arrays generated
The number of unique genus labels: 954


### Encoding the labels into a one-hot-encoded format at Species level

In [12]:
taxon = 'Species'
taxon_dict = get_taxon_dict(test_na, taxon)[0]

y_train_spe_na = to_categorical(y=train_na[taxon].map(taxon_dict).astype(np.float32))
np.save(f'arrays/species/{nameof(y_train_spe_na)}', y_train_spe_na)

# y_train_spe_a = to_categorical(y=train_a[taxon].map(taxon_dict).astype(np.float32))
# np.save(f'arrays/species/{nameof(y_train_spe_a)}', y_train_spe_a)

y_test_spe_na = to_categorical(y=test_na[taxon].map(taxon_dict).astype(np.float32))
np.save(f'arrays/species/{nameof(y_test_spe_na)}', y_test_spe_na)

labelsval_spe_na = to_categorical(y=val_na[taxon].map(taxon_dict).astype(np.float32))
np.save(f'arrays/species/{nameof(labelsval_spe_na)}', labelsval_spe_na)

# labelsval_spe_a = to_categorical(y=val_a[taxon].map(taxon_dict).astype(np.float32))
# np.save(f'arrays/species/{nameof(labelsval_spe_a)}', labelsval_spe_a)
print('Species label arrays generated')
# ------------------------------------------------------------------------------------
spe_count = train_na[taxon].nunique()
print(f'The number of unique species labels: {spe_count}')

Species train/test/val arrays generated
The number of unique species labels: 1569


## Loading the generated training/validation/testing arrays
When seperating the processing and training/evaluation scripts, due to memory limits, the saved arrays can be loaded using this script.

In [ ]:
# AMOUNT OF UNIQUE LABELS AT EACH TAXON LEVEL
fam_count, gen_count, spe_count = 349, 954, 1569

# LOADING ENCODED SEQUENCES for the CNN models in 2 processing variations
# FOR CNN  |  with 4-mer
x_train_CNN_na4 = np.load(f'arrays/CNN/x_train_CNN_na4.npy')
# x_train_CNN_a4 = np.load(f'arrays/CNN/x_train_CNN_a4.npy')
x_test_CNN_na4 = np.load(f'arrays/CNN/x_test_CNN_na4.npy')
dataval_CNN_na4 = np.load(f'arrays/CNN/dataval_CNN_na4.npy')
# dataval_CNN_a4 = np.load(f'arrays/CNN/dataval_CNN_a4.npy')
print('4-mers LOADED')
# -----------------------------------------------------------------------------
# FOR CNN  |  with 7-mer
x_train_CNN_na7 = np.load(f'arrays/CNN/x_train_CNN_na7.npy')
x_train_CNN_a7 = np.load(f'arrays/CNN/x_train_CNN_a7.npy')
x_test_CNN_na7 = np.load(f'arrays/CNN/x_test_CNN_na7.npy')
dataval_CNN_na7 = np.load(f'arrays/CNN/dataval_CNN_na7.npy')
dataval_CNN_a7 = np.load(f'arrays/CNN/dataval_CNN_a7.npy')
print('7-mers LOADED')
# -----------------------------------------------------------------------------
print('CNN sequences LOADED')

###############################################################################

# LOADING ENCODED SEQUENCES for the RNN models in 2 processing variations
# FOR RNN  |  with regular one-hot-encoding
x_train_RNN_na0 = np.load(f'arrays/CNN/x_train_RNN_na0.npy')
# x_train_RNN_a0 = np.load(f'arrays/CNN/x_train_RNN_a0.npy')
x_test_RNN_na0 = np.load(f'arrays/CNN/x_test_RNN_na0.npy')
dataval_RNN_na0 = np.load(f'arrays/CNN/dataval_RNN_na0.npy')
# dataval_RNN_a0 = np.load(f'arrays/CNN/dataval_RNN_a0.npy')
print('Regular one-hot-encoded sequences LOADED')
# -----------------------------------------------------------------------------
# FOR RNN  |  with matation rate adjusted one-hot-encoding
x_train_RNN_na1 = np.load(f'arrays/CNN/x_train_RNN_na1.npy')
x_train_RNN_a1 = np.load(f'arrays/CNN/x_train_RNN_a1.npy')
x_test_RNN_na1 = np.load(f'arrays/CNN/x_test_RNN_na1.npy')
dataval_RNN_na1 = np.load(f'arrays/CNN/dataval_RNN_na1.npy')
dataval_RNN_a1 = np.load(f'arrays/CNN/dataval_RNN_a1.npy')
print('Mutation rate adjusted one-hot-encoded sequences LOADED')
# -----------------------------------------------------------------------------
print('RNN sequences LOADED')

###############################################################################

# LOADING one-hot encoded labels at each taxon level
# -----------------------------------------------------------------------------
# LABELS AT FAMILY LEVEL
y_train_fam_na = np.load(f'arrays/CNN/y_train_fam_na.npy')
# y_train_fam_a = np.load(f'arrays/CNN/y_train_fam_a.npy')
y_test_fam_na = np.load(f'arrays/CNN/y_test_fam_na.npy')
labelsval_fam_na = np.load(f'arrays/CNN/labelsval_fam_na.npy')
# labelsval_fam_a = np.load(f'arrays/CNN/labelsval_fam_a.npy')
print('Family label arrays LOADED')
# -----------------------------------------------------------------------------
# LABELS AT GENUS LEVEL
y_train_gen_na = np.load(f'arrays/CNN/y_train_gen_na.npy')
y_train_gen_a = np.load(f'arrays/CNN/y_train_gen_a.npy')
y_test_gen_na = np.load(f'arrays/CNN/y_test_gen_na.npy')
labelsval_gen_na = np.load(f'arrays/CNN/labelsval_gen_na.npy')
labelsval_gen_a = np.load(f'arrays/CNN/labelsval_gen_a.npy')
print('Genus label arrays LOADED')
# -----------------------------------------------------------------------------
# LABELS AT SPECIES LEVEL
y_train_spe_na = np.load(f'arrays/CNN/y_train_spe_na.npy')
# y_train_spe_a = np.load(f'arrays/CNN/y_train_spe_a.npy')
y_test_spe_na = np.load(f'arrays/CNN/y_test_spe_na.npy')
labelsval_spe_na = np.load(f'arrays/CNN/labelsval_spe_na.npy')
# labelsval_spe_a = np.load(f'arrays/CNN/labelsval_spe_a.npy')
print('Species label arrays LOADED')

## Setting up the network architectures
What follows are a set of functions for creating the deep learning models.

### 1. Convolutional Neural Network (CNN)

In [15]:
# CNN
def make_CNNmodel(input_shape, out_len, name='CNN'):
    CNNmodel = keras.Sequential(
        [
            Reshape(target_shape = input_shape, input_shape = input_shape[:-1]),
            Conv1D(5, 5, padding='valid', input_shape=input_shape),
            Activation('relu'),
            MaxPooling1D(pool_size=2, padding='valid'),

            Conv1D(10, 5, padding='valid'),
            Activation('relu'),
            MaxPooling1D(pool_size=2, padding='valid'),

            Flatten(),
            Dense(500),
            Activation('relu'),
            Dropout(0.5),

            Dense(out_len, activation='softmax')
        ], 
        name = name
    )
    return CNNmodel

### 2. Bidirectional Long-Short Term Memory Neural Network (BiLSTM)

In [16]:
# BiLSTM
def make_BiLSTMmodel(out_len, INPUT_SHAPE=INPUT_SHAPE_RNN, name='BiLSTM'):
    BiLSTMmodel = keras.Sequential(
        [
            Masking(mask_value=0., input_shape=INPUT_SHAPE_RNN),
            Bidirectional(LSTM(128, return_sequences=True), merge_mode='sum'),
            Dropout(0.5),
            AveragePooling1D(4),
            Bidirectional(LSTM(128), merge_mode='sum'),
            Dropout(0.5),
            Dense((out_len), activation='softmax'),
        ],
        name=name
    )
    return BiLSTMmodel

### 3. Convolutional BiLSTM Neural Network (ConvBiLSTM)

In [17]:
# ConvBiLSTM
def make_ConvBiLSTMmodel(out_len, INPUT_SHAPE=INPUT_SHAPE_RNN, name='ConvBiLSTM'):
    ConvBiLSTMmodel = keras.Sequential(
        [
            Masking(mask_value=0., input_shape=INPUT_SHAPE),
                        
            Conv1D(128, 3),
            AveragePooling1D(),
            Dropout(0.2),

            Conv1D(128, 3),
            AveragePooling1D(),
            Dropout(0.2),

            Conv1D(128, 3, use_bias=True),
            AveragePooling1D(),
            Dropout(0.2),
            
            Bidirectional(LSTM(128, activation='tanh'), merge_mode='sum'),
            Dropout(0.2),
            Dense(128, activation='relu'),
            Dropout(0.2),
            Dense(out_len, activation='softmax')
        ], 
        name = name
    )
    return ConvBiLSTMmodel

### 4. Attention-based ConvBiLSTM (Read2Pheno)

In [18]:
# Read2Pheno
## Conv & Res net layers
CONV_NET_nr, RES_NET_nr, NET_filters, NET_window = 2, 1, 64, 2
## extra Dropout layer (after Res block)
DROP_r, POOL_s = 0.2, 2
## BiLSTM layer
LSTM_nodes = 128
## attention Layers
ATT_layers,vATT_nodes = 1, 128
## fully connected layers
FC_layers, FC_nodes, FC_drop = 1, 128, 0.3

#####################################################################################################
# BLOCK FUNCTIONS
def conv_net_block(X, n_cnn_filters=256, cnn_window=9, block_name='convblock'):
    '''
    convolutional block with a 1D convolutional layer, a batch norm layer followed by a relu activation.
    parameters:
        n_cnn_filters: number of output channels
        cnn_window: window size of the 1D convolutional layer
    '''
    X = Conv1D(n_cnn_filters, cnn_window, strides=1, padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    return X

def res_net_block(X, n_cnn_filters=256, cnn_window=9, block_name='resblock'):
    '''
    residual net block accomplished by a few convolutional blocks.
    parameters:
        n_cnn_filters: number of output channels
        cnn_window: window size of the 1D convolutional layer
    '''
    X_identity = X
    # cnn0
    X = Conv1D(n_cnn_filters, cnn_window, strides=1, padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    # cnn1
    X = Conv1D(n_cnn_filters, cnn_window, strides=1, padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    # cnn2
    X = Conv1D(n_cnn_filters, cnn_window, strides=1, padding='same')(X)
    X = BatchNormalization()(X)
    X = Add()([X, X_identity])
    X = Activation('relu')(X)
    return X

def attention_layer(H_lstm, n_layer, n_node, block_name='att'):
    '''
    feedforward attention layer accomplished by time distributed dense layers.
    parameters:
        n_layer: number of hidden layers
        n_node: number of hidden nodes
    '''
    H_emb = H_lstm
    for i in range(n_layer):
        H_lstm = TimeDistributed(Dense(n_node, activation="tanh"))(H_lstm)
    M = TimeDistributed(Dense(1, activation="linear"))(H_lstm)
    alpha = keras.layers.Softmax(axis=1)(M)
    r_emb = Dot(axes = 1)([alpha, H_emb])
    r_emb = Flatten()(r_emb)
    return r_emb

def fully_connected(r_emb, n_layer, n_node, drop_out_rate=0.5, block_name='fc'):
    '''
    fully_connected layer consists of a few dense layers.
    parameters:
        n_layer: number of hidden layers
        n_node: number of hidden nodes
        drop_out_rate: dropout rate to prevent the model from overfitting
    '''
    for i in range(n_layer):
        r_emb = Dense(n_node, activation="relu")(r_emb)
    r_emb = Dropout(drop_out_rate)(r_emb) 
    return r_emb
    
#####################################################################################################
# TOTAL MODEL FUNCTION

def make_R2Pmodel(out_len, INPUT_SHAPE=INPUT_SHAPE_RNN, name='Read2Pheno'):
    X = Input(shape=INPUT_SHAPE)
    X_mask = Masking(mask_value=0.)(X)

    ## CONV Layers
    X_cnn = X_mask
    # conv_net
    for i in range(CONV_NET_nr):
        X_cnn = conv_net_block(X_cnn, n_cnn_filters=NET_filters, cnn_window=NET_window)
    # res_net
    for i in range(RES_NET_nr):
        X_cnn = res_net_block(X_cnn, n_cnn_filters=NET_filters, cnn_window=NET_window)

    ## Extra Pooling layer and Dropout
    X_pool = AveragePooling1D(pool_size=POOL_s)(X_cnn)
    X_drop = Dropout(DROP_r)(X_pool)

    ## RNN Layers
    H_lstm = Bidirectional(LSTM(LSTM_nodes, return_sequences=True), merge_mode='sum')(X_drop)
    H_lstm = Activation('tanh')(H_lstm)

    ## ATT Layers
    r_emb = attention_layer(H_lstm, n_layer=ATT_layers, n_node=ATT_nodes, block_name = 'att')
        
    # Fully connected layers
    r_emb = fully_connected(r_emb, n_layer=FC_layers, n_node=FC_nodes, drop_out_rate=FC_drop, block_name = 'fc')

    # Compile model
    out = Dense(out_len, activation='softmax', name='final_dense')(r_emb)
    R2Pmodel = Model(inputs = X, outputs = out, name = name)
    
    return R2Pmodel

## Creating the Deep Learning models
The models are generated based on input and output shapes.

### 1. CNN
The CNN models are created, tailored to the different input (k-mer) and output (taxon) shapes. The input is determined by the k-mer used and the output shape is determined by the amount of unique taxon labels.

In [19]:
# for Family
CNN_fam_4 = make_CNNmodel(input_shape=INPUT_SHAPE_4_MER, out_len=fam_count, name='CNN_Family-level_4-mer') # with 4-mer
CNN_fam_7 = make_CNNmodel(input_shape=INPUT_SHAPE_7_MER, out_len=fam_count, name='CNN_Family-level_7-mer') # with 7-mer
# for Genus
CNN_gen_4 = make_CNNmodel(input_shape=INPUT_SHAPE_4_MER, out_len=gen_count, name='CNN_Genus-level_4-mer')
CNN_gen_7 = make_CNNmodel(input_shape=INPUT_SHAPE_7_MER, out_len=gen_count, name='CNN_Genus-level_7-mer')
# for Species
CNN_spe_4 = make_CNNmodel(input_shape=INPUT_SHAPE_4_MER, out_len=spe_count, name='CNN_Species-level_4-mer')
CNN_spe_7 = make_CNNmodel(input_shape=INPUT_SHAPE_7_MER, out_len=spe_count, name='CNN_Species-level_7-mer')

### 2. RNN
The RNN models are created, tailored to the different output shapes. The output shape is determined by the amount of unique taxon labels.

In [20]:
# for Family
BiLSTM_fam = make_BiLSTMmodel(out_len=fam_count, name='BiLSTM_Family-level')
ConvBiLSTM_fam = make_ConvBiLSTMmodel(out_len=fam_count, name='ConvBiLSTM_Family-level')
R2P_fam = make_R2Pmodel(out_len=fam_count, name='Read2Pheno_Family-level')
# for Genus
BiLSTM_gen = make_BiLSTMmodel(out_len=gen_count, name='BiLSTM_Genus-level')
ConvBiLSTM_gen = make_ConvBiLSTMmodel(out_len=gen_count, name='ConvBiLSTM_Genus-level')
R2P_gen = make_R2Pmodel(out_len=gen_count, name='Read2Pheno_Genus-level')
# for Species
BiLSTM_spe = make_BiLSTMmodel(out_len=spe_count, name='BiLSTM_Species-level')
ConvBiLSTM_spe = make_ConvBiLSTMmodel(out_len=spe_count, name='ConvBiLSTM_Species-level')
R2P_spe = make_R2Pmodel(out_len=spe_count, name='Read2Pheno_Species-level')

## Compiling, Training and Evaluating the Deep Learning models

For each model:
- a Weights and Biases run is initiated
- the model is compiled and a summary is printed
- the model is trained and training-time is measured
- the model is evaluated by calculating the test-loss and -accuracy, the F1 score and the MCC score
- the training history and metrics are saved

In [23]:
def train_and_evaluate_model(model, train_data, train_labels, validation_data, validation_labels, test_data, test_labels):
    wandb.init(project = 'Test training2', entity = 'bachelorprojectgroup9', name=model.name)

    print (f'Loading {model.name} model...')
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=LR), metrics=['accuracy'])
    print(model.summary())

    print (f'Fitting {model.name} model...')
    start_time = time.time()
    history = model.fit(train_data, train_labels, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data = (validation_data, validation_labels), callbacks=[WandbCallback()])
    time_taken = round(time.time() - start_time)
    np.save(f'history/{model.name}_history.npy', history.history)
    
    print (f'Evaluating {model.name} model...')
    test_labels_arg = np.argmax(test_labels, axis=1)
    test_predictions = np.argmax(model.predict(test_data), axis=1)
    loss, accuracy = model.evaluate(test_data, test_labels)

    # F1-score: harmonic mean of the precision and recall
    #   score from 0 to 1
    f1 = f1_score(y_true=test_labels_arg, y_pred=test_predictions, average='weighted')
    # Matthews correlation coefficient: coefficient of +1 represents a perfect prediction,
    #   0 an average random prediction and -1 an inverse prediction
    mcc = matthews_corrcoef(y_true=test_labels_arg, y_pred=test_predictions)

    score_dict = pd.DataFrame({'Model/run' : model.name, 'Data' : argname('train_data'), 'Training time' : time_taken, 'Test loss' : loss, 'Test accuracy' : accuracy, 'F1-score' : f1, 'MCC' : mcc}, index=[0])
    print(score_dict)
    score_dict.to_csv(f'scores/{model.name}_evaluation', index=False)

    wandb.finish()
    return history, score_dict

In [ ]:
# RUNNING CNN MODELS
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# running the model at genus level with both k-mers
CNN_gen_4_history, CNN_gen_4_score_dict = train_and_evaluate_model(CNN_gen_4, x_train_CNN_na4, y_train_gen_na, dataval_CNN_na4, labelsval_gen_na, x_test_CNN_na4, y_test_gen_na)
CNN_gen_7_history, CNN_gen_7_score_dict = train_and_evaluate_model(CNN_gen_7, x_train_CNN_na7, y_train_gen_na, dataval_CNN_na7, labelsval_gen_na, x_test_CNN_na7, y_test_gen_na)
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# running the 7-mer model at family and species level
CNN_fam_7_history, CNN_fam_7_score_dict = train_and_evaluate_model(CNN_fam_7, x_train_CNN_na7, y_train_fam_na, dataval_CNN_na7, labelsval_fam_na, x_test_CNN_na7, y_test_fam_na)
CNN_spe_7_history, CNN_spe_7_score_dict = train_and_evaluate_model(CNN_spe_7, x_train_CNN_na7, y_train_spe_na, dataval_CNN_na7, labelsval_spe_na, x_test_CNN_na7, y_test_spe_na)
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# running the 7-mer model at genus level on the augmented data
CNN_gen_7a_history, CNN_gen_7a_score_dict = train_and_evaluate_model(CNN_gen_7, x_train_CNN_a7, y_train_gen_a, dataval_CNN_a7, labelsval_gen_na, x_test_CNN_na7, y_test_gen_na)

###################################################################################################################################################################################################

# RUNNING RNN MODELS
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# running the models at genus level with both one-hot-encodings
BiLSTM_gen_0_history, BiLSTM_gen_0_score_dict = train_and_evaluate_model(BiLSTM_gen, x_train_RNN_na0, y_train_gen_na, dataval_RNN_na0, labelsval_gen_na, x_test_RNN_na0, y_test_gen_na)
ConvBiLSTM_gen_0_history, ConvBiLSTM_gen_0_score_dict = train_and_evaluate_model(ConvBiLSTM_gen, x_train_RNN_na0, y_train_gen_na, dataval_RNN_na0, labelsval_gen_na, x_test_RNN_na0, y_test_gen_na)
R2P_gen_0_history, R2P_gen_0_score_dict = train_and_evaluate_model(R2P_gen, x_train_RNN_na0, y_train_gen_na, dataval_RNN_na0, labelsval_gen_na, x_test_RNN_na0, y_test_gen_na)

BiLSTM_gen_history, BiLSTM_gen_score_dict = train_and_evaluate_model(BiLSTM_gen, x_train_RNN_na1, y_train_gen_na, dataval_RNN_na1, labelsval_gen_na, x_test_RNN_na1, y_test_gen_na)
ConvBiLSTM_gen_history, ConvBiLSTM_gen_score_dict = train_and_evaluate_model(ConvBiLSTM_gen, x_train_RNN_na1, y_train_gen_na, dataval_RNN_na1, labelsval_gen_na, x_test_RNN_na1, y_test_gen_na)
R2P_gen_1_history, R2P_gen_1_score_dict = train_and_evaluate_model(R2P_gen, x_train_RNN_na1, y_train_gen_na, dataval_RNN_na1, labelsval_gen_na, x_test_RNN_na1, y_test_gen_na)
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# running the models at family and species level with the mutation rate adjusted one-hot-encoding
BiLSTM_fam_history, BiLSTM_fam_score_dict = train_and_evaluate_model(BiLSTM_fam, x_train_RNN_na1, y_train_fam_na, dataval_RNN_na1, labelsval_fam_na, x_test_RNN_na1, y_test_fam_na)
ConvBiLSTM_fam_history, ConvBiLSTM_fam_score_dict = train_and_evaluate_model(ConvBiLSTM_fam, x_train_RNN_na1, y_train_fam_na, dataval_RNN_na1, labelsval_fam_na, x_test_RNN_na1, y_test_fam_na)
R2P_fam_history, R2P_fam_score_dict = train_and_evaluate_model(R2P_fam, x_train_RNN_na1, y_train_fam_na, dataval_RNN_na1, labelsval_fam_na, x_test_RNN_na1, y_test_fam_na)

BiLSTM_spe_history, BiLSTM_spe_score_dict = train_and_evaluate_model(BiLSTM_spe, x_train_RNN_na1, y_train_spe_na, dataval_RNN_na1, labelsval_spe_na, x_test_RNN_na1, y_test_spe_na)
ConvBiLSTM_spe_history, ConvBiLSTM_spe_score_dict = train_and_evaluate_model(ConvBiLSTM_spe, x_train_RNN_na1, y_train_spe_na, dataval_RNN_na1, labelsval_spe_na, x_test_RNN_na1, y_test_spe_na)
R2P_spe_history, R2P_spe_score_dict = train_and_evaluate_model(R2P_spe, x_train_RNN_na1, y_train_spe_na, dataval_RNN_na1, labelsval_spe_na, x_test_RNN_na1, y_test_spe_na)
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# running the models at family level with the mutation rate adjusted one-hot-encoding on the augmented data
BiLSTM_gen_a_history, BiLSTM_gen_a_score_dict = train_and_evaluate_model(BiLSTM_gen, x_train_RNN_a1, y_train_gen_a, dataval_RNN_a1, labelsval_gen_a, x_test_RNN_na1, y_test_gen_na)
ConvBiLSTM_gen_a_history, ConvBiLSTM_gen_a_score_dict = train_and_evaluate_model(ConvBiLSTM_gen, x_train_RNN_a1, y_train_gen_a, dataval_RNN_a1, labelsval_gen_a, x_test_RNN_na1, y_test_gen_na)
R2P_gen_a_history, R2P_gen_a_score_dict = train_and_evaluate_model(R2P_gen, x_train_RNN_a1, y_train_gen_a, dataval_RNN_a1, labelsval_gen_a, x_test_RNN_na1, y_test_gen_na)

## Ribosomal Database Project (RDP)
Train and validation dataset are merged into one, this is becuase the RDP machine learning model does not require validation steps during training. The databases are converted to ready4train taxonomy and fasta files to be fed into the RDP Classifier.

When running cells, new files will be saved in "RDPfiles" directory at the same folder of this jupyter notebook is located. Directory name can be changed.

### 1. Setting variables
Training and classification is done with the whole taxon hierarchy, from kingdom to species, but a specific level can be chosen during evaluation.

In [5]:
global RDPfiles

#variables
RDPfiles = "RDPfiles"
classifier_loc = "rdptools/classifier.jar"
confidence_score = 0.8
level = 'genus' #ranks = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']

### 2. Processing functions
**Lineage2taxTrain** converts tab separated taxonomy text files into the ready4train_taxonomy.txt file. This text file contains the hierarchical taxonomy information in the following format: tax ID | taxon name | parent taxid | depth | rank.
- Tax ID is the index of the rank in the taxonomy file
- Taxon name is the name for the taxonomic rank
- Parent taxid is the tax ID of the rank above the current rank
- Depth is the depth of the rank. Depth 0 is always root. The kingdom rank has depth of 1
- Rank is the taxonomic ranks

**AddFullLineage** generates the ready4train_seqs.fasta file. It has structure similar to the sequence fasta file, but semicolon separated taxonomy is added next to sequence ID.


**RDPoutput2score** generates the accuracy, F1 score and MCC using output.txt and test_taxonomy.txt. Output.txt is generated when classifying test_sequences.txt with the trained model.

In [3]:
def lineage2taxTrain(raw_taxons):
    taxons_list = raw_taxons.strip().split('\n')
    header = taxons_list[0].split('\t')[1:] # headers = list of ranks
    hash = {} # taxon name-id map
    ranks = {} # column number-rank map
    lineages = [] # list of unique lineages

    with open("{}/ready4train_taxonomy.txt".format(RDPfiles), "w") as f:
        # initiate root rank taxon id map
        hash = {"Root":0}
        for i in range(len(header)):
            name = header[i]
            ranks[i] = name

        # root rank info
        root = ['0', 'Root', '-1', '0', 'rootrank']
        f.write("*".join(root) +  '\n')

        ID = 0
        for line in taxons_list[1:]:
            cols = line.strip().split('\t')[1:]
            # iterate each column
            for i in range(len(cols)):
                name = []
                for node in cols[:i + 1]:
                    node = node.strip()
                    if not node in ('-', ''):
                        name.append(node)

                pName = ";".join(name[:-1])
                if not name in lineages:
                    lineages.append(name)

                depth = len(name)
                name = ';'.join(name)
                if name in hash.keys():
                    # already seen this lineage
                    continue
                try:
                    rank = ranks[i]
                except KeyError:
                    print (cols)
                    sys.exit()

                if i == 0:
                    pName = 'Root'
                # parent taxid
                pID = hash[pName]
                ID += 1
                # add name-id to the map
                hash[name] = ID
                out = ['%s'%ID, name.split(';')[-1], '%s'%pID, '%s'%depth, rank]
                f.write("*".join(out) + '\n')
    f.close()

def addFullLineage(raw_taxons, raw_seqs):
    # lineage map
    hash = {}
    taxonomy_list = raw_taxons.strip().split('\n')

    for line in taxonomy_list[1:]:
        line = line.strip()
        cols = line.strip().split('\t')
        lineage = ['Root']

        for node in cols[1:]:
            node = node.strip()
            if not (node == '-' or node == ''):
                lineage.append(node)

        ID = cols[0]
        lineage = ';'.join(lineage).strip()
        hash[ID] = lineage

    sequence_list = raw_seqs.strip().split('\n')
    with open("{}/ready4train_seqs.fasta".format(RDPfiles), "w") as f:
        for line in sequence_list:
            line = line.strip()
            if line == '':
                continue
            if line[0] == '>':
                ID = line.strip().split()[0].replace('>', '')
                lineage = hash[ID]
                f.write('>' + ID + '\t' + lineage + '\n')
            else:
                f.write(line.strip() + '\n')
    f.close()

def RDPoutput2score(pred_file, true_file, level, cf):
    taxon_list = []
    ranks = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
    level = ranks.index(level)

    pred = pd.read_csv(pred_file, sep="\t", header=None)
    pred.drop(pred.columns[1:level+5+2*level],  axis = 'columns', inplace=True)
    pred.drop(pred.columns[4:], axis = 'columns', inplace=True)
    
    pred_dict = {}
    for index, row in pred.iterrows():
        row = row.tolist()
        if row[1] not in taxon_list:
            taxon_list += [row[1]]
        if float(row[3]) >= cf:
            pred_dict[row[0]] = row[1]

    true = pd.read_csv(true_file, sep="\t", header=None)
    true_dict = {}
    for index, row in true.iterrows():
        true_dict[row[0]] = row[level+1]
        if row[level+1] not in taxon_list:
            taxon_list += [row[level+1]]


    y_pred, y_true = [], []
    for i in pred_dict.keys():
        y_pred.append(taxon_list.index(pred_dict[i]))
        y_true.append(taxon_list.index(true_dict[i]))

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted') # works well if both y_true_decode and y_pred are list
    mcc = matthews_corrcoef(y_true, y_pred)

    print("accuracy: {}\nF1-score: {}\nMCC-score: {}".format(acc, f1, mcc))

## Main
In this cell, a new directory is made to save output.

- Train and validation sets are merged into one training set.
- raw_seqs + raw_taxons -> ready4train files (input for training the RDP model)
- test dataset -> test_sequcnes.fasta + test_taxonomy.txt (used in evaluation of new trained models)

In [43]:
# main_RDP.py
os.system("mkdir {}".format(RDPfiles))

# merge train and validation dataframes into one.
train = pd.concat([train_na, val_na], ignore_index=True)
# train = pd.concat([train_a, val_a], ignore_index=True)
test = test_na

# convert train and test dataframe into tab separated taxonomy and sequence string
# taxnomy file is converted to a tab separated string
# sequence file is converted to fasta format with sequence ID and sequence
raw_seqs = ''
raw_taxons = 'SeqId Kingdom	Phylum	Class	Order	Family	Genus	Species' + '\n'
for index, row in train.iterrows():
    taxons = row.tolist()
    raw_seqs += '>' + taxons[0] + '\n' + taxons[-1] + '\n'
    raw_taxons += '\t'.join(taxons[:-1]) + '\n'

# convert test dataframe into text and fasta files to be utilized by RDP
# taxnomy file is converted to a tab separated text file
# sequence file is converted to fasta format
with open("{}/test_sequences.fasta".format(RDPfiles), "w") as seq_f, open("{}/test_taxonomy.txt".format(RDPfiles), "w") as tax_f:
    for index, row in test.iterrows():
        taxons = row.tolist()
        seq_f.write('>' + taxons[0] + '\n' + taxons[-1] + '\n')
        tax_f.write('\t'.join(taxons[:-1]) + '\n')
    seq_f.close()
    tax_f.close()

# convert raw taxonomy and sequence files to ready4rdp trainable files
lineage2taxTrain(raw_taxons)
addFullLineage(raw_taxons, raw_seqs)

print("Data preprocessing for RDP completed")

Data preprocessing for RDP completed


Training is performed with the train() function. The models is saved in training_files directory with four weight files. New file, rRNAClassifier.properties, are necessary for bridging these files and the RDP Classifier. Classification is performed with the classify() function. Option -o leads RDP Classifier to use the newly generated training models.

Using RDPoutput2score, the accuracy, F1 score and MCC are calculated. These values will be used to compare with the deep learning models. User can choose specific level of taxon to evaluate prediction.

In [6]:
# Training the RDP classifier
start_time = time.time()
os.system("java -Xmx10g -jar {} train -o {}/training_files -s {}/ready4train_seqs.fasta -t {}/ready4train_taxonomy.txt".format(classifier_loc, RDPfiles, RDPfiles, RDPfiles))
with open("{}/training_files/rRNAClassifier.properties".format(RDPfiles), "w") as f:
    f.write("bergeyTree=bergeyTrainingTree.xml" + '\n' + "probabilityList=genus_wordConditionalProbList.txt" + '\n' + "probabilityIndex=wordConditionalProbIndexArr.txt" + '\n' + "wordPrior=logWordPrior.txt" + '\n' + "classifierVersion=RDP Naive Bayesian rRNA Classifier Version 2.5, May 2012 ")
    f.close()
time_taken = round(time.time() - start_time)
print("RDP training-time: {} seconds".format(time_taken))

# Testing the RDP classifier
os.system("java -Xmx10g -jar {} classify -t {}/training_files/rRNAClassifier.properties  -o {}/output.txt {}/test_sequences.fasta".format(classifier_loc, RDPfiles, RDPfiles, RDPfiles))

# Evaluating the RDP classifier
RDPoutput2score("{}/output.txt".format(RDPfiles), "{}/test_taxonomy.txt".format(RDPfiles), level, confidence_score)

RDP training-time: 85 seconds
accuracy: 0.9724941724941725
F1-score: 0.9726149488062451
MCC-score: 0.9723559927778965


## Visualizing the results